# Generate the Paranal Colorized Dataset

* The dataset is composed by merging all individual cases
* The color or clustering of logtext is applied on every row of the dataset
* The variable ```config.THRESHOLD_MIN_AVG_COUNT``` is set to discard those very rare events.




In [38]:
import sys
sys.path.append("../../")

import pandas as pd
from src.colors import color_paranal as color
from src import config

path="../../data/interim/PARANAL/"

In [61]:
def merge_datasets(FILES, origin=""):
    dataset=[]
    for f in FILES:
        instance=pd.read_csv(path+f)
        instance['instance_id'] = f
        if len(dataset) == 0:
            dataset = instance
        else:
            dataset = pd.concat([instance, dataset])
    dataset['color'] = dataset['logtext'].apply(color)
    dataset['source'] = origin + dataset['source']
    
    return dataset


## Collect OBS

In [62]:
FILES=!ls $path | grep OBS

In [83]:
# Merge all FILES in one
obs_df = merge_datasets(FILES, "OBS-")

# Extract unique colors and count/total
num_instances = obs_df['instance_id'].nunique()
obs_colors = obs_df.groupby("color")["@timestamp"].count() / num_instances

# Add this new column to the dataset
obs_df['color_count'] = obs_df.color.apply( lambda r:obs_colors[r] )

# Filter by Threshold THRESHOLD_MIN_AVG_COUNT
obs_df_filtered = obs_df[ obs_df['color_count'] > config.THRESHOLD_MIN_AVG_COUNT ]

In [84]:
obs_df[['color', 'source', 'instance_id']].describe()

,color,source,instance_id
count,125800,125800,125800
unique,1058,14,142
top,{} springgreen4,OBS-wgv/bob_42025,GRAVITY-2018-12-16T02:46:41.104Z.OBS.csv
freq,7835,23754,2168


In [85]:
obs_df_filtered[['color', 'source', 'instance_id']].describe()

,color,source,instance_id
count,123789,123789,123789
unique,643,14,142
top,{} springgreen4,OBS-wgv/bob_42025,GRAVITY-2018-12-16T02:46:41.104Z.OBS.csv
freq,7835,23550,2045


In [112]:
obs_df_filtered.head(5)

,@timestamp,source,logtext,instance_id,color,color_count
0,2018-12-30T00:37:09.554Z,OBS-wgv/bob_884,OB started at 2018-12-30T00:37:09,GRAVITY-2018-12-30T00:37:09.554Z.OBS.csv,ob started _date_,1.000000
1,2018-12-30T00:37:09.554Z,OBS-wgv/bob_884,(OBS.NAME: M05_HD15875_MED_SPLIT // OBS.ID: 2...,GRAVITY-2018-12-30T00:37:09.554Z.OBS.csv,obs name {} obs id {},0.964789
2,2018-12-30T00:37:09.571Z,OBS-wgv/bob_884,GRAVITY_single_acq -- GRAVITY Single Mode Acq...,GRAVITY-2018-12-30T00:37:09.554Z.OBS.csv,gravity_single_acq gravity single mode acquisi...,0.908451
3,2018-12-30T00:37:09.572Z,OBS-wgv/bob_884,Started at 2018-12-30T00:37:09 (underlined),GRAVITY-2018-12-30T00:37:09.554Z.OBS.csv,started _date_ underlined,1.767606
4,2018-12-30T00:37:09.834Z,OBS-wgv/bob_884,COU AG START = 'F',GRAVITY-2018-12-30T00:37:09.554Z.OBS.csv,cou ag start f,0.901408


## Collect ATs

In [87]:
at_x=[]
for telescope in ["AT1", "AT2", "AT3", "AT4"]:
    # Merge all FILES in one
    FILES=!ls $path | grep $telescope
    at_x.append( merge_datasets(FILES, "%s-" % telescope) )

at_df = pd.concat(at_x)
del(at_x)

# Extract unique colors and count/total
num_instances = at_df['instance_id'].nunique()
at_colors = at_df.groupby("color")["@timestamp"].count() / num_instances

# Add this new column to the dataset
at_df['color_count'] = at_df.color.apply( lambda r:at_colors[r] )

# Filter by Threshold THRESHOLD_MIN_AVG_COUNT
at_df_filtered = at_df[ at_df['color_count'] > config.THRESHOLD_MIN_AVG_COUNT ]

In [88]:
at_df[['color', 'source', 'instance_id']].describe()

,color,source,instance_id
count,348629,348629,348629
unique,1438,236,568
top,ins opti1 move motion execution,AT1-wat1tcs/nmmgrControl,GRAVITY-2018-12-16T04:07:59.099Z.AT3.csv
freq,18639,99375,6284


In [89]:
at_df_filtered[['color', 'source', 'instance_id']].describe()

,color,source,instance_id
count,342788,342788,342788
unique,386,155,568
top,ins opti1 move motion execution,AT1-wat1tcs/nmmgrControl,GRAVITY-2018-12-16T04:07:59.099Z.AT3.csv
freq,18639,98274,5490


In [113]:
at_df_filtered.head()

,@timestamp,source,logtext,instance_id,color,color_count
0,2018-12-30T00:37:10.000Z,AT1-wat1tcs/logManager,INS.OPTI1.MOVE Motion execution.,GRAVITY-2018-12-30T00:37:09.554Z.AT1.csv,ins opti1 move motion execution,32.815141
1,2018-12-30T00:37:13.000Z,AT1-wat1tcs/logManager,INS.OPTI2.MOVE Motion execution.,GRAVITY-2018-12-30T00:37:09.554Z.AT1.csv,ins opti2 move motion execution,32.808099
2,2018-12-30T00:37:13.000Z,AT1-wat1tcs/logManager,INS.OPTI1.MOVEDONE Motor offset done.,GRAVITY-2018-12-30T00:37:09.554Z.AT1.csv,ins opti1 movedone motor offset done,32.808099
3,2018-12-30T00:37:13.466Z,AT1-wat1tcs/atifControl,"TIF : Received command: SETGVCM, Buffer: 0,...",GRAVITY-2018-12-30T00:37:09.554Z.AT1.csv,tif received command setgvcm buffer {} {} {} {...,1.603873
4,2018-12-30T00:37:13.468Z,AT1-wat1tcs/atifControl,TIF : Succesfully completed command: SETGVC...,GRAVITY-2018-12-30T00:37:09.554Z.AT1.csv,tif succesfully completed command setgvcm buff...,1.603873


## Save into CSV

In [91]:
path = "../../data/processed/PARANAL/"
!mkdir -p $path

In [108]:
at_df_filtered[[
    "@timestamp", "source", "logtext", "instance_id", "color"
]].to_csv(path+'dataset_at_filtered.csv', index=False)

In [105]:
at_colors.to_csv(path+'dataset_at_colors.csv', header=False, index=True)

In [109]:
obs_df_filtered[[
    "@timestamp", "source", "logtext", "instance_id", "color"
]].to_csv(path+'dataset_obs_filtered.csv', index=False)

In [110]:
obs_colors.to_csv(path+'dataset_obs_colors.csv', header=False, index=True)

In [111]:
!ls -ltrh $path | grep dataset

-rw-r--r--  1 jgil  5000   135K Jan 12 22:04 dataset_at_colors.csv
-rw-r--r--  1 jgil  5000    68M Jan 12 22:04 dataset_at_filtered.csv
-rw-r--r--  1 jgil  5000    22M Jan 12 22:05 dataset_obs_filtered.csv
-rw-r--r--  1 jgil  5000    91K Jan 12 22:05 dataset_obs_colors.csv
